In [252]:
import tweepy
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from langdetect import detect

 
# import twitter_credentials
import numpy as np
import pandas as pd
consumer_key = "cFf7im7BH68xO9qh3zEsv3nFz"
consumer_secret = "2QyodEVK63XYE5D9RFPAo0I53rhBOsNocQGpGB8rapmEqxDnJi"
access_token = "1265319352795975680-EJBIU55ZHZnjn8svR420cqVuU9evRL"
access_token_secret = "i8h2utF6b2l2Uh7Vpg6c2mnPOO1CuthGdbyWtHSWdIURP"



# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
    
# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        return auth

# # # # TWITTER STREAMER # # # #

# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)


class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])
        df['Date Created'] = np.array([tweet.created_at for tweet in tweets])
        df["24H"] = df["Date Created"].apply(lambda x: 0 if (datetime.utcnow()-x)>timedelta(1) else 1)
        df.drop(columns=["Date Created"],axis=1,inplace=True)
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df.drop(df[df['likes']<=1000].index,inplace=True)
        df.drop(columns="likes",inplace=True)

        return df
class FilterData():
    
     def filter_RT():
        df["Retweets"] = df["Tweets"].apply(lambda x: True if x[:2]=="RT" else False)
        df.drop(df[df["Retweets"]==True].index,inplace=True)
        df.drop(columns=["Retweets"],axis=1,inplace=True)
        df.index = [i for i in range(df["Tweets"].count())]
     def filter_lang():
        df["Lang"] = df["Tweets"].apply(lambda x: False if detect(x)=="en" else True)
        df.drop(df[df["Lang"]==True].index,inplace=True)
        df.drop(columns=["Lang"],axis=1,inplace=True)
        df.index = [i for i in range(df["Tweets"].count())]

        
if __name__ == '__main__':

    twitter_client = TwitterClient()
    
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="narendramodi", count=200)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    FilterData.filter_RT()
    FilterData.filter_lang()
    df.to_csv('modi-tweets.csv')
df

,Tweets,24H
0,India and the Philippines will cooperate to re...,1
1,Had a useful exchange with President Rodrigo D...,1
2,Shri Ved Marwah Ji will be remembered for his ...,0
3,"Rwanda is, and will remain, an important pilla...",0
4,"Thank you, my friend President @PaulKagame. \n...",0
...,...,...
160,Excellent effort by @CNNnews18. They have wond...,0
161,Discussed COVID-19 pandemic with good friend P...,0
162,Please thank Swetha for this creative and affe...,0
163,Good gesture by the @DelhiPolice.\n\nOur Gurud...,0


In [253]:
urls=[]
def filter_url(x):
        if "https://" in x:
            index = x.find("https://")
            urls.append(x[index:])
        else:
            urls.append("0")
df["Tweets"].apply(filter_url)
df["URL"] = urls

In [254]:
df

,Tweets,24H,URL
0,India and the Philippines will cooperate to re...,1,https://t.co/WDn16d4Biz
1,Had a useful exchange with President Rodrigo D...,1,https://t.co/sAkeSHDCKE
2,Shri Ved Marwah Ji will be remembered for his ...,0,https://t.co/uUyZ58iBIy
3,"Rwanda is, and will remain, an important pilla...",0,0
4,"Thank you, my friend President @PaulKagame. \n...",0,https://t.co/9jrdBpmLcC
...,...,...,...
160,Excellent effort by @CNNnews18. They have wond...,0,https://t.co/VqUJZZ78lM
161,Discussed COVID-19 pandemic with good friend P...,0,https://t.co/zJhncRcJOy
162,Please thank Swetha for this creative and affe...,0,https://t.co/XqtliBFlgW
163,Good gesture by the @DelhiPolice.\n\nOur Gurud...,0,https://t.co/qPRkYD2Knt


In [255]:
# from selenium import webdriver
# def tweet_extraction(x):
#     driver=webdriver.Chrome(executable_path='C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe') #the path where chromedriver.exe is installed in your local system
#     driver.get(x)
#     ele=driver.find_elements_by_xpath('css-1dbjc4n')
#     for r in ele:
#         print(r.text.strip())
#     driver.close()

# RUN THE BELOW CELL IF YOU ARE CRAZY

In [256]:
# for i in s:
#     if i!="0":
#         tweet_extraction(i[:i.find(" ")])
# #         '"'+i[:i.find(" ")]+'"'

In [257]:
# from selenium import webdriver
# import time
# link = ''
# driver=webdriver.Chrome(executable_path='C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe') #the path where chromedriver.exe is installed in your local system
# driver.get(link)
# time.sleep(5)
# ele=driver.find_elements_by_xpath('css-1dbjc4n')
# for i in ele:
#     print(i.text)
# driver.close()

In [258]:
# from selenium import webdriver
# import time
# link = 'https://t.co/WDn16d4Biz'
# driver = webdriver.Chrome(executable_path='C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe') #the path where chromedriver.exe is installed in your local system
# driver.get(link)
# time.sleep(10)
# elements = []
# for i in range(2, 4):
#     try:
#         elements.append(driver.find_elements_by_xpath('/html/body/div/div/div/div[2]/main/div/div/div/div/div/div/div/section/div/div/div/div[' + str(i) + ']/div/div/div/div/article/div/div[2]/div[2]/div[2]/div[1]/div/span'))
#     except:
#         continue
# for i in elements:
#     for j in i:
#         print(j.text)
# driver.close()

In [259]:
df

,Tweets,24H,URL
0,India and the Philippines will cooperate to re...,1,https://t.co/WDn16d4Biz
1,Had a useful exchange with President Rodrigo D...,1,https://t.co/sAkeSHDCKE
2,Shri Ved Marwah Ji will be remembered for his ...,0,https://t.co/uUyZ58iBIy
3,"Rwanda is, and will remain, an important pilla...",0,0
4,"Thank you, my friend President @PaulKagame. \n...",0,https://t.co/9jrdBpmLcC
...,...,...,...
160,Excellent effort by @CNNnews18. They have wond...,0,https://t.co/VqUJZZ78lM
161,Discussed COVID-19 pandemic with good friend P...,0,https://t.co/zJhncRcJOy
162,Please thank Swetha for this creative and affe...,0,https://t.co/XqtliBFlgW
163,Good gesture by the @DelhiPolice.\n\nOur Gurud...,0,https://t.co/qPRkYD2Knt


In [269]:
s = df["Tweets"]
lists = ["#COVID-19","#COVID19","COVID-19","CORONA","CORONAINDIA","#COVID19INDIA","COVID19INDIA"]
count = 0
for i in s:
    for j in lists:
        if j in i:
            count += 1
            print(i)
            print(count)

Had a useful exchange with President Rodrigo Duterte about COVID-19 and other issues.  I thanked him for taking car… https://t.co/sAkeSHDCKE
1
Thank you, my friend President @PaulKagame. 

Rwanda has managed the COVID-19 crisis effectively under your leaders… https://t.co/9jrdBpmLcC
2
Had an excellent talk with H.E. Filipe Nyusi, President of Mozambique on COVID-19 situation. I assured him of India… https://t.co/i3LdEb1o80
3
In this difficult time of COVID-19 pandemic, wishing Armenian PM @NikolPashinyan and his family a speedy recovery.… https://t.co/Q4kGzq7yXX
4
Thank you, Prime Minister @PKJugnauth for our warm conversation today! Congratulations for successfully controlling COVID-19 in Mauritius.
5
Had an excellent talk with President @GotabayaR.  Sri Lanka is fighting COVID-19 effectively under his leadership.… https://t.co/teEjcKD3LI
6
Spoke to PM @Statsmin about the COVID-19 pandemic, and how India and Denmark can strengthen cooperation in the post-COVID era.
7
The post COVID-19